## arxiv and wikipedia tools with RAG

In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [3]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)

#creating an wikipedia tool
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki

WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Harsh Vora\\OneDrive\\Desktop\\PROJECTS\\agenticai\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250))

In [4]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)

#creating an arxiv tool
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv

ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250))

In [5]:
#loading rag tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [7]:
import os
from dotenv import load_dotenv

os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [8]:
loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001B093762330>, search_kwargs={})

### converting the retriever into a tool

In [9]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever, "langsmith search", "search any info regarding langsmith")

In [10]:
tools=[wiki, arxiv, retriever_tool]

In [11]:
os.environ["GROQUER_API_KEY"]=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
llm=ChatGroq(model_name="Llama3-8b-8192")
llm          

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B0D073EFF0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B0D075C140>, model_name='Llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain import hub                             
prompt=hub.pull("hwchase17/openai-functions-agent")
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [14]:
### Agents

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm, tools, prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [15]:
### Agent executor
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [16]:
agent_executor.invoke({"input":"tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith search` with `{'query': 'langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

Skip to main conten

{'input': 'tell me about langsmith',
 'output': 'Based on the results from the Langsmith search tool call, it appears that Langsmith is a platform for building production-grade Large Language Model (LLM) applications. It provides features such as observability, evaluation, and prompt engineering to help developers closely monitor and evaluate their applications, iterate on prompts, and ship quickly with confidence.\n\nSome of the key features of Langsmith include:\n\n* Observability: Langsmith provides LLM-native observability, allowing developers to get meaningful insights from their application. This includes tracing, metrics, dashboards, and alerts.\n* Evaluation: Langsmith allows developers to evaluate their application over production traffic, scoring application performance and getting human feedback on their data.\n* Prompt Engineering: Langsmith provides tools for prompt engineering, including automatic version control and collaboration features.\n\nIt seems that Langsmith is a

In [17]:
agent_executor.invoke({"input":"tell me about indias independence history"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': "India's independence movement"}`


Page: Indian independence movement
Summary: The Indian independence movement was a series of historic events in South Asia with the ultimate aim of ending British colonial rule. It lasted until 1947, when the Indian Independence Act 1947 was passed.

Invoking: `wikipedia` with `{'query': 'British colonial rule'}`


Page: British Empire
Summary: The British Empire comprised the dominions, colonies, protectorates, mandates, and other territories ruled or administered by the United Kingdom and its predecessor states. It began with the overseas possessions and trad
Invoking: `wikipedia` with `{'query': 'Indian independence movement'}`


Page: Indian independence movement
Summary: The Indian independence movement was a series of historic events in South Asia with the ultimate aim of ending British colonial rule. It lasted until 1947, when the Indian Independence Act 1947 was pa

{'input': 'tell me about indias independence history',
 'output': 'The Indian independence movement was a significant event in the country\'s history. Here\'s a summary:\n\nThe Indian independence movement, which lasted for about a century, was a series of events and movements that aimed to end British colonial rule in India. The movement began with the Indian Rebellion of 1857, which was sparked by widespread discontent among Indians against British rule. The rebellion was crushed, but it marked the beginning of a long struggle for independence.\n\nOver the years, various movements and protests were organized to demand independence. Mahatma Gandhi\'s non-violent resistance movement, which began in the 1920s, was a major turning point in the struggle. Gandhi\'s philosophy of non-violence and civil disobedience inspired many Indians to participate in the movement, and his leadership helped to bring about a unified response to British rule.\n\nThe struggle for independence gained momentu

In [18]:
agent_executor.invoke({"input":"tell me attention is all you need"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'attention is all you need'}`


Page: Attention Is All You Need
Summary: "Attention Is All You Need" is a 2017 landmark research paper in machine learning authored by eight scientists working at Google. The paper introduced a new deep learning architecture known as the transformer,
Invoking: `arxiv` with `{'query': 'transformer neural machine translation'}`


Published: 2022-11-28
Title: Summer: WeChat Neural Machine Translation Systems for the WMT22 Biomedical Translation Task
Authors: Ernan Li, Fandong Meng, Jie Zhou
Summary: This paper introduces WeChat's participation in WMT 2022 shared biomedical
tra
Invoking: `langsmith search` with `{'query': 'WeChat neural machine translation'}`


Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeploy

{'input': 'tell me attention is all you need',
 'output': "It seems like the user has provided a lot of information about LangSmith, but it doesn't seem directly related to the previous conversation about attention mechanisms in transformers or neural machine translation. I'm going to respond directly as a helpful assistant.\n\nLangSmith seems like an interesting platform for building production-grade LLM applications. I can see how it could be useful for monitoring and evaluating the performance of AI applications. However, I'm not sure how it specifically relates to the original topic of attention mechanisms in transformers or neural machine translation. If you'd like to discuss more about LangSmith or any other topic, I'm happy to help!"}

### converting to a react agent

In [19]:
prompt=hub.pull("hwchase17/react")

In [20]:
from langchain.agents import AgentExecutor, create_react_agent
agent=create_react_agent(llm, tools, prompt)

In [21]:
agent_executor=AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
agent_executor.invoke({"input":"tell me about langsmith"})



> Entering new AgentExecutor chain...
Thought: Langsmith seems to be a language model for searching and retrieving information. I should try to use the langsmith search tool to get more information about Langsmith.

Action: langsmith search
Action Input: 'Langsmith'Get started with LangSmith | 🦜️🛠️ LangSmith

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrumentation needed.If you're already using either of these, see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmi

In [ ]:
agent_executor.invoke({"input":"tell me about irish independence"})

In [ ]:
agent_executor.invoke({"input":"tell me about latest research in earth science from rice university"})